# **Extracción ofertas Infojobs**

En este notebook se desarrolla el código para la extracción de la información de los puestos de trabajo en Madrid de : Data science, Data Analyst y Data Engineer.

Se realiza un scrapeo con Selenium en el cual nos loggeamos con un usuario y contraseña en la plataforma e interactuamos como si furamos un humano para evitar así los captchas.

Los datos recogidos se van almacenando en un `csv` dentro de la carpeta `datos_crudo`.

In [2]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import sys
sys.path.append("../")

from src import funciones_infojobs as f_infojobs

load_dotenv()

True

A continación se define la función principal la cual ejecuta los siguientes pasos: 
- Accede a la página principal y acepta las cookies si las hubiera.
- Avanza hasta el inicio de sesión y accede con los datos personales.
- Para cada empleo ejecuta un bucle en el que:
    - Navega directamente hasta la url proporcionada en el diccionario para dicho empleo.
    - Recoge la información para cada página, entrando en cada oferta, obteniendo la información deseada y almacenándola.
    - Comprueba si hay más páginas y en caso de haberla avanza a la siguiente.

In [4]:
f_infojobs.extraccion_ofertas_infojobs()

Cookies aceptadas

 ------------------------BUSCANDO OFERTAS PARA data_engineer---------------------------


 *** 
 Página 1, filas df: 0
-------------------------------------------------------
https://www.infojobs.net/madrid/system-development-engineer-reliability-and-automation-engineering/of-i63f04d87dd4e968df30a289e72400d?applicationOrigin=search-new%7Celement%7E54064376822&searchId=54064376822&page=1&sortBy=RELEVANCE
GUARDAR
System Development Engineer, Reliability and Automation Engineering
Amazon Europe Core
Madrid, Madrid (España)
Presencial
Publicada hace 2d (Publicada de nuevo)
Salario no disponible
Experiencia mínima: al menos 1 año
Tipo de contrato: indefinido, jornada completa
INSCRIBIRME EN ESTA OFERTA
Datos de la oferta 1 guardados correctamente.
-------------------------------------------------------
https://www.infojobs.net/madrid/database-engineer/of-i940a39a5684d4d991088847422fdfb?applicationOrigin=search-new%7Celement%7E54064376822&searchId=54064376822&page=1&sortBy